In [211]:
global word, block, rounds
word = 8 #each word has 4 bytes
block = 64 #each block is set to 64 bytes, i.e., 512 bits
rounds = 10

# Whirlpool Cipher Transformation Functions

In [212]:
SubBytes = (
    0x18, 0x23, 0xC6, 0xE8, 0x87, 0xB8, 0x01, 0x4F, 0x36, 0xA6, 0xD2, 0xF5, 0x79, 0x6F, 0x91, 0x52,
    0x60, 0xBC, 0xB,  0x8E, 0xA3, 0x0C, 0x7B, 0x35, 0x1D, 0xE0, 0xD7, 0xC2, 0x2E, 0x4B, 0xFE, 0x57,
    0x15, 0x77, 0x37, 0xE5, 0x9F, 0xF0, 0x4A, 0xCA, 0x58, 0xC9, 0x29, 0x0A, 0xB1, 0xA0, 0x6B, 0x85,
    0xBD, 0x5D, 0x10, 0xF4, 0xCB, 0x3E, 0x05, 0x67, 0xE4, 0x27, 0x41, 0x8B, 0xA7, 0x7D, 0x95, 0xC8,
    0xFB, 0xEE, 0x7C, 0x66, 0xDD, 0x17, 0x47, 0x9E, 0xCA, 0x2D, 0xBF, 0x07, 0xAD, 0x5A, 0x83, 0x33,
    0x63, 0x02, 0xAA, 0x71, 0xC8, 0x19, 0x49, 0xC9, 0xF2, 0xE3, 0x5B, 0x88, 0x9A, 0x26, 0x32, 0xB0,
    0xE9, 0x0F, 0xD5, 0x80, 0xBE, 0xCD, 0x34, 0x48, 0xFF, 0x7A, 0x90, 0x5F, 0x20, 0x68, 0x1A, 0xAE,
    0xB4, 0x54, 0x93, 0x22, 0x64, 0xF1, 0x73, 0x12, 0x40, 0x08, 0xC3, 0xEC, 0xDB, 0xA1, 0x8D, 0x3D,
    0x97, 0x00, 0xCF, 0x2B, 0x76, 0x82, 0xD6, 0x1B, 0xB5, 0xAF, 0x6A, 0x50, 0x45, 0xF3, 0x30, 0xEF,
    0x3F, 0x55, 0xA2, 0xEA, 0x65, 0xBA, 0x2F, 0xC0, 0xDE, 0x1C, 0xFD, 0x4D, 0x92, 0x75, 0x06, 0x8A,
    0xB2, 0xE6, 0x0E, 0x1F, 0x62, 0xD4, 0xA8, 0x96, 0xF9, 0xC5, 0x25, 0x59, 0x84, 0x72, 0x39, 0x4C,
    0x5E, 0x78, 0x38, 0x8C, 0xC1, 0xA5, 0xE2, 0x61, 0xB3, 0x21, 0x9C, 0x1E, 0x43, 0xC7, 0xFC, 0x04,
    0x51, 0x99, 0x6D, 0x0D, 0xFA, 0xDF, 0x7E, 0x24, 0x3B, 0xAB, 0xCE, 0x11, 0x8F, 0x4E, 0xB7, 0xEB,
    0x3C, 0x81, 0x94, 0xF7, 0xB9, 0x13, 0x2C, 0xD3, 0xE7, 0x6E, 0xC4, 0x03, 0x56, 0x44, 0x7F, 0xA9,
    0x2A, 0xBB, 0xC1, 0x53, 0xDC, 0x0B, 0x9D, 0x6C, 0x31, 0x74, 0xF6, 0x46, 0xAC, 0x89, 0x14, 0xE1,
    0x16, 0x3A, 0x69, 0x09, 0x70, 0xB6, 0xC0, 0xED, 0xCC, 0x42, 0x98, 0xA4, 0x28, 0x5C, 0xF8, 0x86,
)
print(int('0x60',16))
round = 1
first = (round-1)*8
second = first+8
SubBytes[first:second]



96


(24, 35, 198, 232, 135, 184, 1, 79)

In [213]:
def subBytes (state):
    for i in range(word):
        for j in range(word):
            state[i][j] = SubBytes[state[i][j]]
    return state

In [214]:
def ShiftColumns(state):
    Nb = len(state)
    n = [word[:] for word in state]
    for i in range(Nb):
        for j in range(word):
            n[i][j] = state[(i-j) % Nb][j]
    return n

In [215]:
constant = [
[1,1,4,1,8,5,2,9],
[9,1,1,4,1,8,5,2],
[2,9,1,1,4,1,8,5],
[5,2,9,1,1,4,1,8],
[8,5,2,9,1,1,4,1],
[1,8,5,2,9,1,1,4],
[4,1,8,5,2,9,1,1],
[1,4,1,8,5,2,9,1]]

def column(matrix, i):
    return [row[i] for row in matrix]

print(column(constant, 4))



[8, 1, 4, 1, 1, 9, 2, 5]


In [216]:
import numpy as np

def MixRows(state):
    res = [[0 for x in range(word)] for y in range(word)] 
    n = int('0x11D',16) #The modulus used by Whirlpool from the textbook
    for i in range(word):
        for j in range (word):
            c = column(constant, j)
            r = state[i][:]
            v = np.mod (np.dot(r,c), n) 
            res[i][j] = v
    return res


In [217]:
def AddRoundKey(state, RC):
    n = 256
    for i in range(word):
        for j in range(word):
            state[i][j] = state[i][j] ^ RC[i][j]
            #print("AddRoundKey:",np.mod(state[i][j], n), n)
            state[i][j] = np.mod(state[i][j], n)
    return state, RC

# Key Expansion Operations for Whirlpool

In [218]:
def RoundConstants(r):
    k = [[0 for x in range(word)] for y in range(word)] 
    first = (r-1)*8
    second = first+8 
    k[0][:] = SubBytes[first:second]
    return k


Generate Round Key (On-demand)

In [219]:
def GenerateRoundKey (key, r):
    key = subBytes(key)
    key = ShiftColumns(key)
    key = MixRows(key)
    RC = RoundConstants(r)
    key, RC = AddRoundKey(key, RC)
    return key

# Whirlpool Cipher Rounds

In [220]:
def whirlpool(state, cipherkey):
    state, key = AddRoundKey(state, cipherkey)
    print("Initial: ",state)
    #Rounds
    for i in range(1, rounds+1):
        print("---------------------CIPHER ROUND", i, "------------------------")
        state = subBytes(state)
        state = ShiftColumns(state)
        state = MixRows(state)
        key = GenerateRoundKey(key, i)
        state, key = AddRoundKey(state, key)
        print("State: ",state)
    return state

# Whirlpool Hash Function

## Reading File

In [221]:
def listToString(s): 
    str1 = "" 
    for ele in s: 
        str1 += ele  
    return str1 

def openFile(filename):
    with open(filename, 'r') as reader:
        lines = reader.read().splitlines()
        print(lines)
    message = listToString(lines)
    print(message)
    byteMessage = bytes(message, 'utf-8')
    state = list(byteMessage)
    print(state)
    return state 

In [222]:
state = openFile("Clean Case.txt")

['vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'water', '']
vinegarpaintthinneroildetergentpowderwatertissuevinegarpaintthinneroildetergentpowderwatertissuevinegarpaintthinneroildetergentpowderwatertissuevinegarpaintthinneroildetergentpowderwatertissuewater
[118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97

## Padding the original message to get odd number of 256-bit blocks 

In [223]:
def padding(message):
    n = len(message)
    temp = []
    if int(n / 32) % 2 == 0:
        #Odd number of blocks with padding the remaining bytes
        #print((((n / 32) % 2) * 32))
        pad = int(((n / 32) % 2) * 32)
        print("Padding required for the remaining bytes: ", pad)
        nBytesToPad = 32 - pad
        print("Bytes to pad:", nBytesToPad)
        for i in range (nBytesToPad):
            if i == 0:
                temp.append(128)
            else:
                temp.append(0)
        state = message + temp
        print(state)
    print("Number of 256-bit blocks in original message after padding: ", int(len(state)/32))
    return state

In [224]:
state = padding(state)

Padding required for the remaining bytes:  5
Bytes to pad: 27
[118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115,

## Add the message length block to create even number of 256-bit blocks

In [225]:
def addMessageLength (state):
    temp = []
    n = len(state)
    print(n)
    for i in range (32):
            if i == 0:
                temp.append(n)
            else:
                temp.append(0)
    state = state + temp
    print (state)
    return state

In [226]:
state = addMessageLength(state)

224
[118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 119, 97, 116, 101, 114, 128, 0, 0, 0, 0, 0, 0, 

In [227]:
print(len(state))
print("Number of 256-bit blocks in original message after Message Length added: ", int(len(state)/32))

256
Number of 256-bit blocks in original message after Message Length added:  8


## Whirlpool hashing rounds functions

In [228]:
def initializeDigest(state):    
    digest = [[0] * 8] * 8 
    return digest

In [229]:
def getState (r, state):
    k = [[0 for x in range(word)] for y in range(word)] 
    j = 0
    for i in range(r*8,r*8+8):
        first = (i)*8
        second = first+8
        #print(first, second)
        #print(state[first:second])
        #print(j)
        k[j][:] = state[first:second]
        j = j + 1
    return k

#print(state)
#digest = getState(3,state)
#print(digest)

In [230]:
def XOR (m1, m2, m3):
    k2 =  [[0 for j in range(0, 8)] for i in range(0, 8)]
    for i in range(8):
        for j in range(8):
            k2[i][j] = m1[i][j] ^ m2[i][j] ^ m3[i][j] 
    return k2

In [231]:
#main hashing function
def whirlpoolhash(state):
    digest = initializeDigest(state)       
    rounds = int ((len(state)/block))
    print ("Hash rounds: ",rounds)   
    print("******************************** Hash Rounds Begin ********************************")
    for i in range(rounds):
        prevdigest = digest
        print("==================================================================")
        print("======================== HASH ROUND", i+1, "============================")
        print("==================================================================")
        messagewindow = getState(i, state)
        print ("Message: ", messagewindow)
        digest = whirlpool(messagewindow, digest)
        print("==============================XOR================================")
        digest = XOR (digest, prevdigest, messagewindow)
        print("Digest: ", digest)
        print("******************************** End of Hash Rounds ********************************")
    return digest

digest = whirlpoolhash(state)
print("Final Digest: ", digest)

Hash rounds:  4
******************************** Hash Rounds Begin ********************************
======================== HASH ROUND 1 ============================
Message:  [[118, 105, 110, 101, 103, 97, 114, 112], [97, 105, 110, 116, 116, 104, 105, 110], [110, 101, 114, 111, 105, 108, 100, 101], [116, 101, 114, 103, 101, 110, 116, 112], [111, 119, 100, 101, 114, 119, 97, 116], [101, 114, 116, 105, 115, 115, 117, 101], [118, 105, 110, 101, 103, 97, 114, 112], [97, 105, 110, 116, 116, 104, 105, 110]]
Initial:  [[118, 105, 110, 101, 103, 97, 114, 112], [97, 105, 110, 116, 116, 104, 105, 110], [110, 101, 114, 111, 105, 108, 100, 101], [116, 101, 114, 103, 101, 110, 116, 112], [111, 119, 100, 101, 114, 119, 97, 116], [101, 114, 116, 105, 115, 115, 117, 101], [118, 105, 110, 101, 103, 97, 114, 112], [97, 105, 110, 116, 116, 104, 105, 110]]
---------------------CIPHER ROUND 1 ------------------------
State:  [[178, 229, 242, 73, 23, 134, 224, 121], [223, 32, 101, 101, 50, 72, 37, 44], [1

## Convert digest list to string

In [232]:
from itertools import chain
def to_str(digest):
    temp = list(chain.from_iterable(digest))
    s = [str(i) for i in temp]
    res = int("".join(s))
    return(res)

In [233]:
stringdig = to_str(digest)

print(digest)
print(stringdig)

[[89, 203, 91, 60, 27, 72, 217, 134], [67, 77, 114, 166, 85, 198, 52, 15], [102, 189, 248, 40, 8, 126, 16, 240], [245, 153, 38, 230, 235, 79, 34, 241], [35, 194, 35, 241, 56, 32, 4, 207], [26, 66, 118, 65, 214, 120, 99, 51], [179, 190, 51, 63, 104, 40, 60, 213], [195, 45, 226, 127, 184, 69, 59, 181]]
8920391602772217134677711416685198521510218924840812616240245153382302357934241351943524156324207266611865214120995117919051631044060213195452261271846959181


## Save hash/digest in a file

In [234]:
def saveDigest(digest):
    f = open("Digest.txt", "w")
    f.write(str(digest))
    f.close()

In [235]:
saveDigest(stringdig)

## Read saved hash from file

In [236]:
def readHash ():
    with open('Digest.txt', 'r') as reader:
        lines = reader.read()
    return lines

In [237]:
saveddigest = readHash()
print(saveddigest)

8920391602772217134677711416685198521510218924840812616240245153382302357934241351943524156324207266611865214120995117919051631044060213195452261271846959181


## Verification of a hash from saved file and a generated hash of a given file

In [238]:
def verification(file, savedhash):
    print (file)
    state = openFile(file)
    state = padding(state)
    state = addMessageLength(state)
    digest = whirlpoolhash(state)
    digest = to_str(digest)
    saveddigest = int(savedhash)
    print("Digest from read file: ", digest)
    print("Digest from save file: ", saveddigest)
    if digest == saveddigest:
        return True
    return False

In [239]:
test = verification('Clean Case.txt', saveddigest)

Clean Case.txt
['vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'vinegar', 'paintthinneroil', 'detergentpowder', 'water', 'tissue', 'water', '']
vinegarpaintthinneroildetergentpowderwatertissuevinegarpaintthinneroildetergentpowderwatertissuevinegarpaintthinneroildetergentpowderwatertissuevinegarpaintthinneroildetergentpowderwatertissuewater
[118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 97, 114, 112, 97, 105, 110, 116, 116, 104, 105, 110, 110, 101, 114, 111, 105, 108, 100, 101, 116, 101, 114, 103, 101, 110, 116, 112, 111, 119, 100, 101, 114, 119, 97, 116, 101, 114, 116, 105, 115, 115, 117, 101, 118, 105, 110, 101, 103, 9

In [240]:
print("Verification Result - Hash Equal? ==> ", test)

Verification Result - Hash Equal? ==>  True


# Test Functions (Ignore)

In [241]:
#testWhirlPoolRounds
from random import randint

def testWhirlRounds():
    count = 1
    state = [[0 for i in range(word)] for j in range(word)] 
    for i in range(word):
        for j in range(word):
            state[i][j] = count
            count += 1
    key = [[0 for i in range(word)] for j in range(word)] 
    for i in range(word):
        for j in range(word):
            key[i][j] = randint(0, 100)
    state = whirlpool(state, key)

testWhirlRounds()

Initial:  [[43, 87, 35, 94, 73, 94, 91, 27], [84, 48, 64, 18, 36, 62, 55, 39], [86, 53, 53, 44, 2, 80, 0, 121], [58, 32, 45, 68, 74, 35, 15, 107], [27, 112, 48, 24, 9, 4, 1, 61], [53, 45, 21, 77, 20, 62, 75, 12], [26, 39, 37, 32, 26, 11, 83, 96], [89, 27, 34, 18, 89, 51, 4, 107]]
---------------------CIPHER ROUND 1 ------------------------
State:  [[136, 245, 230, 187, 173, 199, 103, 250], [5, 92, 166, 61, 14, 93, 211, 33], [174, 16, 255, 166, 231, 206, 17, 79], [241, 105, 239, 87, 88, 13, 54, 153], [202, 71, 17, 74, 93, 242, 97, 112], [125, 218, 191, 140, 40, 148, 0, 63], [125, 35, 199, 102, 234, 47, 172, 170], [117, 88, 50, 223, 27, 175, 14, 31]]
---------------------CIPHER ROUND 2 ------------------------
State:  [[0, 253, 95, 198, 69, 3, 249, 172], [99, 77, 199, 146, 24, 101, 4, 98], [34, 75, 185, 6, 121, 121, 7, 185], [129, 246, 161, 221, 123, 206, 51, 173], [52, 91, 160, 255, 197, 91, 127, 135], [2, 180, 88, 134, 218, 239, 118, 19], [16, 199, 216, 118, 60, 195, 55, 79], [45, 109,

In [164]:
#testGenerateRoundKey
def testGenRKey():
    count = 1
    state = [[0 for i in range(word)] for j in range(word)] 
    for i in range(word):
        for j in range(word):
            state[i][j] = count
            count += 1
    print(state)
    key = GenerateRoundKey(state, 1)
    print(key)
testGenRKey()

[[1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 39, 40], [41, 42, 43, 44, 45, 46, 47, 48], [49, 50, 51, 52, 53, 54, 55, 56], [57, 58, 59, 60, 61, 62, 63, 64]]
[[208, 18, 227, 102, 181, 77, 94, 156], [127, 196, 253, 135, 10, 165, 23, 221], [249, 26, 148, 242, 68, 134, 205, 191], [7, 155, 89, 17, 89, 76, 38, 149], [35, 9, 58, 93, 1, 19, 129, 5], [111, 13, 15, 43, 16, 99, 244, 176], [248, 30, 123, 198, 202, 11, 240, 219], [108, 125, 238, 40, 148, 18, 248, 128]]


In [165]:
#test RoundConstants
RoundConstants(1)

[[24, 35, 198, 232, 135, 184, 1, 79],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [15]:
#test subbytes
print(int('0x18',16))
print(SubBytes[91])
def testsub():
    count = 1
    state = [[0 for i in range(word)] for j in range(word)] 
    for i in range(word):
        for j in range(word):
            state[i][j] = count
            count += 1
    print(state)
    state = subBytes(state)
    print(state)
testsub()

24
136
[[1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 39, 40], [41, 42, 43, 44, 45, 46, 47, 48], [49, 50, 51, 52, 53, 54, 55, 56], [57, 58, 59, 60, 61, 62, 63, 64]]
[[35, 198, 232, 135, 184, 1, 79, 54], [166, 210, 245, 121, 111, 145, 82, 96], [188, 11, 142, 163, 12, 123, 53, 29], [224, 215, 194, 46, 75, 254, 87, 21], [119, 55, 229, 159, 240, 74, 202, 88], [201, 41, 10, 177, 160, 107, 133, 189], [93, 16, 244, 203, 62, 5, 103, 228], [39, 65, 139, 167, 125, 149, 200, 251]]


In [210]:
#testshiftcol
def testShiftCol():
    count = 1
    state = [[0 for i in range(word)] for j in range(word)] 
    for i in range(word):
        for j in range(word):
            state[i][j] = count
            count += 1
    print(state)
    state = ShiftColumns(state)
    print(state)
testShiftCol()

[[1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 39, 40], [41, 42, 43, 44, 45, 46, 47, 48], [49, 50, 51, 52, 53, 54, 55, 56], [57, 58, 59, 60, 61, 62, 63, 64]]
[[1, 58, 51, 44, 37, 30, 23, 16], [9, 2, 59, 52, 45, 38, 31, 24], [17, 10, 3, 60, 53, 46, 39, 32], [25, 18, 11, 4, 61, 54, 47, 40], [33, 26, 19, 12, 5, 62, 55, 48], [41, 34, 27, 20, 13, 6, 63, 56], [49, 42, 35, 28, 21, 14, 7, 64], [57, 50, 43, 36, 29, 22, 15, 8]]


In [211]:
#test mixrows
def testmix():
    state = [[1, 58, 51, 44, 37, 30, 23, 16], [9, 2, 59, 52, 45, 38, 31, 24], [17, 10, 3, 60, 53, 46, 39, 32], [25, 18, 11, 4, 61, 54, 47, 40], [33, 26, 19, 12, 5, 62, 55, 48], [41, 34, 27, 20, 13, 6, 63, 56], [49, 42, 35, 28, 21, 14, 7, 64], [57, 50, 43, 36, 29, 22, 15, 8]]
    print(state)
    state = MixRows(state)
    print(state)
testmix()

[[1, 58, 51, 44, 37, 30, 23, 16], [9, 2, 59, 52, 45, 38, 31, 24], [17, 10, 3, 60, 53, 46, 39, 32], [25, 18, 11, 4, 61, 54, 47, 40], [33, 26, 19, 12, 5, 62, 55, 48], [41, 34, 27, 20, 13, 6, 63, 56], [49, 42, 35, 28, 21, 14, 7, 64], [57, 50, 43, 36, 29, 22, 15, 8]]
[[139, 263, 78, 109, 177, 147, 234, 73], [96, 162, 262, 101, 76, 168, 162, 193], [216, 119, 161, 0, 68, 67, 183, 121], [144, 239, 118, 184, 252, 59, 82, 142], [165, 167, 238, 141, 151, 243, 74, 41], [64, 188, 166, 261, 108, 142, 258, 33], [56, 87, 187, 189, 228, 99, 157, 217], [240, 79, 86, 210, 156, 219, 114, 116]]


In [212]:
#testaddround
def testaddroundkey():
    state = [[1, 58, 51, 44, 37, 30, 23, 16], [9, 2, 59, 52, 45, 38, 31, 24], [17, 10, 3, 60, 53, 46, 39, 32], [25, 18, 11, 4, 61, 54, 47, 40], [33, 26, 19, 12, 5, 62, 55, 48], [41, 34, 27, 20, 13, 6, 63, 56], [49, 42, 35, 28, 21, 14, 7, 64], [57, 50, 43, 36, 29, 22, 15, 8]]
    print(state)
    RC = [[24, 35, 198, 232, 135, 184, 1, 79],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0]]
    state = AddRoundKey(state, RC)
    print(state)
testaddroundkey()

[[1, 58, 51, 44, 37, 30, 23, 16], [9, 2, 59, 52, 45, 38, 31, 24], [17, 10, 3, 60, 53, 46, 39, 32], [25, 18, 11, 4, 61, 54, 47, 40], [33, 26, 19, 12, 5, 62, 55, 48], [41, 34, 27, 20, 13, 6, 63, 56], [49, 42, 35, 28, 21, 14, 7, 64], [57, 50, 43, 36, 29, 22, 15, 8]]
[[25, 25, 245, 196, 162, 166, 22, 95], [9, 2, 59, 52, 45, 38, 31, 24], [17, 10, 3, 60, 53, 46, 39, 32], [25, 18, 11, 4, 61, 54, 47, 40], [33, 26, 19, 12, 5, 62, 55, 48], [41, 34, 27, 20, 13, 6, 63, 56], [49, 42, 35, 28, 21, 14, 7, 64], [57, 50, 43, 36, 29, 22, 15, 8]]
